In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git # installing latest version of transformers library
!pip install torch # torch
!pip install peft # necessary for finetuning of the large model via LoRA approach
!pip install bitsandbytes # necessary for quantiziation
!pip install evaluate # extension of the transformers library
!pip install datasets # extension of the transformers library
!pip install accelerate

In [ ]:
from datasets import Dataset as HFDataset, DatasetDict
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

import torch
import pandas as pd
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    DataCollatorWithPadding)
from transformers import AutoModelForCausalLM
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import torch
from peft import get_peft_model
import pandas as pd
from tqdm.auto import tqdm
import bitsandbytes as bnb

import evaluate
import numpy as np

import random
from transformers import pipeline
import json

import os
from google.colab import drive

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


In [ ]:
from transformers import AutoTokenizer, Gemma3ForConditionalGeneration, BitsAndBytesConfig
import logging
from transformers import logging as transformers_logging

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
def load_dataset(clean=True, mount_drive=True):
    """
    Load datasets from JSON files in Google Drive and optionally clean NaN values

    Args:
        clean (bool): If True, remove rows with NaN values in the text column
        mount_drive (bool): If True, mount Google Drive (only needs to be done once per session)

    Returns:
        dict: Dictionary containing train, val, and test dataframes
    """

    if mount_drive:
        drive.mount('/content/drive')

    dataset_folder = '/content/drive/MyDrive/dataset_mental_health'

    train_path = os.path.join(dataset_folder, 'train_data.json')
    val_path = os.path.join(dataset_folder, 'val_data.json')
    test_path = os.path.join(dataset_folder, 'test_data.json')


    with open(train_path, "r") as f:
        train = json.load(f)
    with open(val_path, "r") as f:
        val = json.load(f)
    with open(test_path, "r") as f:
        test = json.load(f)

    train_df = pd.DataFrame(train["data"])
    val_df = pd.DataFrame(val["data"])
    test_df = pd.DataFrame(test["data"])

    if clean:
        train_len_before = len(train_df)
        val_len_before = len(val_df)
        test_len_before = len(test_df)

        # drop NaN values
        train_df = train_df.dropna(subset=['text'])
        val_df = val_df.dropna(subset=['text'])
        test_df = test_df.dropna(subset=['text'])

        train_removed = train_len_before - len(train_df)
        val_removed = val_len_before - len(val_df)
        test_removed = test_len_before - len(test_df)

        print(f"Removed {train_removed} rows with NaN values from training dataset")
        print(f"Removed {val_removed} rows with NaN values from validation dataset")
        print(f"Removed {test_removed} rows with NaN values from testing dataset")
        print(f"Total removed: {train_removed + val_removed + test_removed} rows")

    return {'train': train_df, 'val': val_df, 'test': test_df}

In [ ]:
dataset= load_dataset(clean=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
LABEL_MAP = {
    'neutral': 0,
    'moderate': 1,
    'high-risk': 2
}

EMOTION_INDEX = {
    0: 'neutral',
    1: 'moderate',
    2: 'high-risk'
}

CLASSES = 3
DATACOLUMN = 'text'
LABELCOLUMN = 'my_label'

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

Zero-Shot

Text generation piepline

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enables 4-bit quantization
    bnb_4bit_use_double_quant=True,  # double quantization for potentially higher accuracy (optional)
    bnb_4bit_quant_type="nf4",  # Quantization type (specifics depend on hardware and library)
    bnb_4bit_compute_dtype=torch.bfloat16  # Compute dtype for improved efficiency (optional)
)

In [ ]:
pipeline = pipeline(task="text-generation", model="google/gemma-2-2b", device=0)
pipeline(" ")

In [ ]:
model_id = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

In [ ]:
def classify_text(text, tokenizer, model, device="cuda"):
    prompt = f"""You are a mental health assistant AI.
Given a user's text, classify the emotional distress level as one of the following three categories: neutral, moderate, or high risk.
Respond with only one of these three labels.

Text: {text}
Label:"""

    input_ids = tokenizer(prompt, return_tensors="pt").to(device)

    output = model.generate(
        **input_ids,
        max_new_tokens=10,
        do_sample=False,
        temperature=0.0,
    )

    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Label:" in generated:
        return generated.split("Label:")[-1].strip().split()[0].lower()
    else:
        return "unknown"


In [ ]:
test_df = dataset['test']

In [ ]:
test_df["gemma2_predicted_label"] = test_df["text"].apply(
    lambda x: classify_text(x, tokenizer, model)
)

Using pydantic

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field

In [ ]:
class RiskLabel(BaseModel):
    label: Literal["neutral", "moderate", "high risk"] = Field(
        description="The predicted mental health risk level for the given message."
    )

schema = RiskLabel.schema_json(indent=2)

In [ ]:
def classify_text_structured(text, tokenizer, model, device="cuda"):
    prompt = f"""
You are an AI assistant that classifies mental health risk in user messages.

Output a JSON object matching the following schema:
{schema}

Classify this message:

"{text}"

Respond only with the JSON object.
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    output = model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.0,
        do_sample=False
    )

    generated = tokenizer.decode(output[0], skip_special_tokens=True)

    try:
        # Extract JSON string from the output
        json_str = generated[generated.index("{") : generated.rindex("}") + 1]
        parsed = json.loads(json_str)
        return parsed.get("label", "unknown")
    except Exception as e:
        print(f"Failed to parse JSON: {e}")
        return "unknown"

In [ ]:
test_df["pydantic_gemma2_predicted_label"] = test_df["text"].apply(
    lambda x: classify_text_structured(x, tokenizer, model, device)
)

In [ ]:
test_df['gemma2_predicted_label'] = test_df['gemma2_predicted_label'].replace({
    'high': 'high-risk'
})

In [ ]:
test_df['gemma2_predicted_label_id'] = test_df['gemma2_predicted_label'].map(LABEL_MAP)

In [ ]:


# quick sanity-check for NaNs
print("NaNs in predicted IDs:", test_df['gemma2_predicted_label_id'].isna().sum())
print("NaNs in true IDs:     ", test_df['true_label_id'].isna().sum())

# drop any row that still has a NaN (should be zero now)
mask = ~(test_df['gemma2_predicted_label_id'].isna() | test_df['true_label_id'].isna())
y_pred = test_df.loc[mask, 'gemma2_predicted_label_id'].astype(int)
y_true = test_df.loc[mask, 'true_label_id'].astype(int)

# metrics
print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro-averaged F1: {f1_score(y_true, y_pred, average='macro'):.4f}")
print("\nClassification report:")
print(classification_report(
    y_true, y_pred,
    target_names=['neutral (0)', 'moderate (1)', 'high-risk (2)']
))

print("Confusion matrix:")
print(confusion_matrix(y_true, y_pred))

In [ ]:
test_df['gemma2_predicted_label_id'].isna().sum()

In [ ]:
test_df.to_csv('/content/drive/MyDrive/results/gemma2_predictions.csv', index=False)

Gemma 3 (12B parameters)

In [ ]:
model_id = "google/gemma-3-12b-pt"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

In [ ]:
model_id = "google/gemma-3-4b-it"
access_token = " "

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=access_token
)

In [ ]:
model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    token=access_token
)


In [ ]:
def classify_text(text, tokenizer, model, device="cuda"):

    messages = [
        {"role": "system", "content": "You are a mental health assistant AI."},
        {"role": "user", "content": f"Given the following text, classify the emotional distress level as one of the following three categories: neutral, moderate, or high risk. Respond with only one of these three labels.\nText: {text}"}
    ]

    # template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    #  response
    with torch.inference_mode():
        output = model.generate(
            **inputs,
            top_p=None,
            top_k=None,
            max_new_tokens=10,
            do_sample=False
        )

    input_len = inputs.input_ids.shape[1]
    generated = tokenizer.decode(output[0][input_len:], skip_special_tokens=True)


    lower_generated = generated.lower().strip()
    if "neutral" in lower_generated:
        return "neutral"
    elif "moderate" in lower_generated:
        return "moderate"
    elif "high" in lower_generated or "high risk" in lower_generated or "high-risk" in lower_generated:
        return "high-risk"
    else:
        return "unknown"


In [ ]:
test_df = dataset['test']
test_df["gemma3_predicted_label"] = test_df["text"].apply(
    lambda x: classify_text(x, tokenizer, model)
)

In [ ]:
test_df['gemma3_predicted_label_id'] = test_df['gemma3_predicted_label'].map(LABEL_MAP)

In [ ]:
test_df['true_label_id'] = test_df['my_label'].map(LABEL_MAP)

In [ ]:

print("NaNs in predicted IDs:", test_df['gemma3_predicted_label_id'].isna().sum())
print("NaNs in true IDs:     ", test_df['true_label_id'].isna().sum())

mask = ~(test_df['gemma3_predicted_label_id'].isna() | test_df['true_label_id'].isna())
y_pred = test_df.loc[mask, 'gemma3_predicted_label_id'].astype(int)
y_true = test_df.loc[mask, 'true_label_id'].astype(int)


print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro-averaged F1: {f1_score(y_true, y_pred, average='macro'):.4f}")
print("\nClassification report:")
print(classification_report(
    y_true, y_pred,
    target_names=['neutral (0)', 'moderate (1)', 'high-risk (2)']
))

print("Confusion matrix:")
print(confusion_matrix(y_true, y_pred))

In [ ]:
test_df.to_csv('/content/drive/MyDrive/results/gemma3_predictions.csv', index=False)